1. The roots of $P_3(x) = \dfrac12x(5x^2-3)$ are $x=0,\pm\sqrt{\dfrac{3}{5}}$. The
    base for each points are

    $$
    p_-(x) = \dfrac{5x^2}{6}-\sqrt{\frac35}\dfrac{5x}{6}\\
    p_+(x) = \dfrac{5x^2}{6}+\sqrt{\frac35}\dfrac{5x}{6}\\
    p_0(x) = 1-\frac{5x^2}{3}
    $$

    Coefficients,
    $$
    w_\pm = \dfrac{5}{9}\\
    w_0 = \dfrac{8}{9}
    $$

    For polynomials of order lower than $3$, the interpolation is exact and therefore
    the integral is exact too. For odd orders, we can also easily see that both the
    exact result and the quadrature rule give $0$ from the symmetry of the problem.
    The only order left is $k=4$,

    $$
    I=\frac{2}{5}\\
    Q=2 \cdot \sqrt{\dfrac{3}{5}}^4 \cdot \dfrac{5}{9}=\dfrac{2}{5}=I
    $$

2. The function is `adaptive_g3` in [`integrate.jl`](integrate.jl) which use
    `integrate_g3` in the same file for each single step and returns an
    `AdaptiveResult` that include the integration result as well as the error
    and the number of interval used.

    The integration on the polynomials can be found [below](#Polynomial-integral)

3. The result of the integration for the functions given using the routine above
    can be found [below](#Complex-functions)

4. The result for the third function is clearly off, the reason is that the function
    has a few $0$ at unfortunate points that fools the solver about the accuracy of
    the current result.

    One easy way to get rid of this problem is to devide the inteval randomly (and
    hope that none of them has a unfortunate zero...). See implementation in
    `adaptive_g3_rand` and the result [below](#Randomized)

In [1]:
include("integrate.jl")
import Integrate

In [2]:
immutable F2m{N}
end
call{N}(::F2m{N}, x) = x.^N - x.^2 + 1

call (generic function with 1230 methods)

#### Polynomial integral

In [3]:
for m in 4:8
    println("$m: ", Integrate.adaptive_g3(F2m{m}(), -1.0, 3/4))
end

4: Value: 1.5235026; Error: 0 ; Interval count: 2
5: Value: 1.1390381; Error: 0 ; Interval count: 2
6: Value: 1.4379679; Error: 6.7e-08; Interval count: 16
7: Value: 1.1635558; Error: 2.1e-07; Interval count: 16
8: Value: 1.3954955; Error: 8.4e-07; Interval count: 14


In [4]:
immutable F2_1
end
call(::F2_1, x) = abs(x)

immutable F2_2
end
function call(::F2_2, x)
    x2 = x^2
    (((500x2 - 700) * x2 + 245) * x2 - 3) * sinpi(2x)^2
end

immutable F2_3
end
function call(::F2_3, x)
    if x == 0
        return float(zero(x))
    end
    x^(3 / 4) * sin(1 / x)
end

call (generic function with 1236 methods)

#### Complex functions

In [5]:
for (i, (F, a, b)) in enumerate(((F2_1(), -1., 1.),
                               (F2_1(), -1., 2.),
                               (F2_2(), -1., 1.),
                               (F2_3(), 0., 1.)))
    println("$i: ", Integrate.adaptive_g3(F, a, b))
end

1: Value: 1 ; Error: 0 ; Interval count: 4
2: Value: 2.5; Error: 7.2e-11; Interval count: 32
3: Value: 1.0436948e-16; Error: 3.8e-16; Interval count: 2
4: Value: 0.40702696; Error: 2.1e-07; Interval count: 388652


#### Randomized

In [6]:
Integrate.adaptive_g3_rand(F2_2(), -1, 1)

Value: 7.3667261; Error: 2.7e-07; Interval count: 130